<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Guided Problem Set 3: Fourier Analysis</h1>


<a name='section_3_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P3.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_3_1">P3.1 Frequency Analysis</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_3_1">P3.1 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_3_2">P3.2 The Discrete Fourier Transform</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_3_2">P3.2 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_3_3">P3.3 Spectrogram and Q-Transform</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_3_3">P3.3 Problems</a></td>
    </tr>
</table>

<h3>Learning Objectives</h3>

The goal of this exercise is to prepare you for analyzing data in Fourier space. These exercises will show you how to go from a simple set of frequencies to a full data analysis. As an example, we will use music, but this can easily be applied to any setup in frequency space.

In this problem set we will explore the following objectives:

- Understand the frequency analysis of continuous and discrete time signals
- Visualize the connection between time and frequency space
- Characterize the energy/power carried by different frequencies
- Filter to remove noise from a signal by visualizing its spectral content


<h3>Importing Data (Colab Only)</h3>

If you are in a Google Colab environment, run the cell below to import the data for this notebook. Otherwise, if you have downloaded the course repository, you do not have to run the cell below.

See the source and attribution information below:


>data: data/P03/track.flac <br>
>source: https://freesound.org/s/647536/ <br>
>attribution: Bertsz <br>
>license type: This work is licensed under the Creative Commons 0 License.

In [ ]:
#>>>RUN: P3.0-runcell00

#Importing data:

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/P03' >> .git/info/sparse-checkout
!git pull origin main

<h3>Importing Libraries</h3>

Before beginning, run the cells below to import the relevant libraries for this notebook.

This set of problems uses libraries that are specific playing audio files, which we will use to demonstrate Fourier techniques.

For both the local Juypter and Colab environments, you will need to run the code cells below. 

In [ ]:
#>>>RUN: P3.0-runcell01

!pip install soundfile

In [ ]:
#>>>RUN: P3.0-runcell02

import numpy as np                  #https://numpy.org/doc/stable/
import matplotlib.pyplot as plt     #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import math                         #https://docs.python.org/3/library/math.html
from scipy.io.wavfile import write  #https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: P3.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_3_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P3.1 Frequency Analysis</h2>    

| [Top](#section_3_0) | [Previous Section](#section_3_0) | [Problems](#problems_3_1) | [Next Section](#section_3_2) |


<h3>Motivation: Gravitational Waves/LIGO</h3>

Every massive object that accelerates produces gravitational waves. These gravitational waves can be intuitively understood as "ripples" in space-time that travel at the speed of light. One of the largest sources of continuous gravitational waves are spinning or orbiting massive objects such as neutron stars and black holes. Much can be learned about these objects by studying their gravitational wave signals, so they are the primary focus of the gravitational wave detectors in the US (LIGO). 


Compact Binary Inspiral Gravitational Waves are of particular interest and show up as significant detections in LIGO. These waves are created by one of three sources: 

- Binary Neutron Stars (BNS)
- Binary Black Hole (BBH)
- Neutron Star-Black Hole Binary (NSBH)


<p align="center">
<img alt="interferometer setup" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/ligo.jpg" width="600"/>
</p>

>source: https://www.ligo.caltech.edu/page/ligos-ifo<br>
>attribution: Caltech/MIT/LIGO Lab

When these events occur, they are measured on Earth at one of the LIGO detectors. When the gravitational waves from a cataclysmic merger pass through the earth, they warp space-time, changing the length of a 4km LIGO arm by a thousandth of the diameter of a proton. Amazingly, the LIGO detector is sensitive to these very small changes, and thus can generate strain vs. time data for the gravitation wave.

Strain is the instrument's detected space change within an arm in comparison to the total space (length) of the arm. In the event of a detectable merger, this strain data will form a "gravitational wave signal".

Of course, many other effects can produce small changes in the arm lengths. The goal of the LIGO experimental analysis is to separate (often called "filter") these uninteresting noise signals from the signal, and then extract parameters from the signal itself using a so-called template fit. What this means is that the signal is compared to a prediction (the template) of what it would look like for a particular type of source, for example a binary black hole merger. These templates are adjusted by varying parameters, such as the masses of the two black holes, until the best possible match to the data is found. This is analogous to other fits to data that you have already seen in this course, albeit with a much more complicated "function". These parameters can give insight about the specific event, or even more global properties of the universe itself.

<h3>Frequency Analysis</h3>

<p align="center">
<img alt="gravitational wave strain vs. time" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/strain.png" width="800"/>
</p>

>source: Project 1

Often, the time-series data (strain as a function of time) is too noisy/messy for easy analysis, so instead we can consider the underlying frequency representation of the same information. This allows us to concentrate on the frequencies for which we expect the black hole merger signal to dominate and easily filter out frequencies that come from power lines, earthquake disturbances, or other noise sources.

If we are able to convert the time-series data to frequency-based data, remove (filter out) the noise frequencies, and then convert the remaining signal back to time-series data, we can get a 'cleaner' time-series version of the data. This cleaned (or "filtered") version is then much easier to analyze in further research. 

These translations of time-series data to frequency-based, and vice versa, are called Fourier transforms and inverse Fourier transforms, respectively, and are the main focus of this problem set. 



**Fourier Transform**<br />
*Space or Time Functions* &rarr; *Spatial or Temporal Frequencies* <br />

**Inverse Fourier Transform**<br />
*Spatial or Temporal Frequencies* &rarr; *Space or Time Functions*

<h3>What is meant by "Frequency Space"</h3>

In 1807, Joseph Fourier posited that any periodic signal could be represented by a sum of a particular set of harmonic sinusoids. 

In other words, all signals can be decomposed into elemental sine and cosine components, as follows:

$$f(t) = c_0 + \sum\limits_{k=1}^\infty (c_k\cos(k\omega_o t) + d_k\sin(k\omega_o t))$$


The images below show examples of decomposing a relatively complicated looking distribution into a sum of sinusoidal waves. Note that this example may not show all of the contributions.


<p align="center">
<img  alt="wave decomposition gif" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/decompose.gif" width="800"/>
</p>


>source: https://www.3blue1brown.com/lessons/fourier-transforms<br>
>attribution: 3blue1brown

<p align="center">
<img  alt="wave decomposition time signals" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/recompose.gif" width="800"/>
</p>

>source: https://www.3blue1brown.com/lessons/fourier-transforms<br>
>attribution: 3blue1brown

<h3>Using Euler's Formula for Fourier Decomposition</h3>

The decomposition of $f(t)$ shown above includes both sine and cosine terms. In order to "simplify" the notation, it is common to compress these two together using <a href="https://en.wikipedia.org/wiki/Complex_number" target="_blank">complex numbers</a> and <a href="https://en.wikipedia.org/wiki/Euler%27s_formula" target="_blank">Euler's Formula</a>:

$$e^{ix} = \cos(x) + i\sin(x)$$

One way that complex numbers are often displayed is as a point in a two-dimensional space where the Y axis is the imaginary component and the X axis is the real one. Using the diagram below, we can write:

$$x+yi=|z|\cos(\theta)+i|z|\sin(\theta)=|z|e^{i\theta}$$

where $|z|$ is the magnitude of the complex number, $|z|=\sqrt{x^2+y^2}$.

<p align="center">
<img alt="complex number unit circle" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/plane.png" width="300"/>
</p>

>source: https://commons.wikimedia.org/wiki/File:Argandgaussplane.png<br>
>attribution: Image by LeonardoG, CC-SA-3.0


In the discussion starting in the next section, we will use the $e^{ix}$ notation instead of plain sine and cosine functions.  

Note that a detailed understanding of all the complexities of imaginary numbers (a very bad pun) is not required to complete the material in this course, but at least a basic knowledge would be helpful.


<h3>Visualization</h3>

The following animation shows how a very non-sinusoidal wave form (a triangular shape) can be generated by superimposing successively higher frequency sinusoidal contributions. Since a sine function can be represented as the projection of circular motion onto one of the axes, a sum of sine functions corresponds to adding ever faster circular motions on top of each other (looking closely, you can see that the line in each successive circle rotates faster and faster). In this case, the lengths of the lines (i.e. the radii of the circles) correspond to the complex amplitudes $|z_{k}|$ that combine the coefficients $c_{k}$ or $d_{k}$ in the formula shown above.

<p align="center">
<img alt="unit circle wave decomposition" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/sawtooth.gif" width="800"/>
</p>

>source: https://commons.wikimedia.org/wiki/File:Animation_zum_Verst%C3%A4ndnis_der_Fourierentwicklung.gif<br>
>attribution: PZim, CC BY-SA 3.0 <https://creativecommons.org/licenses/by-sa/3.0>, via Wikimedia Commons


<a name='problems_3_1'></a>     

| [Top](#section_3_0) | [Restart Section](#section_3_1) | [Next Section](#section_3_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.1.1</span>

The code below plots a function in the space or time domain. What will its frequency domain look like? Specifically, how many peaks will it have and will they have the same height?

Answer in the form of a list with the first element indicating how many peaks there are and the second element being 1 if all the peaks are the same height or 0 if not (e.g. `[2,0]` for two peaks with different heights).

*Hint: Look at the function written in the code below. Try plotting this yourself! We will learn how to plot functions in the frequency domain in the next section.*


In [ ]:
#>>>PROBLEM: P3.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0,4*np.pi,0.01)   # start,stop,step
y = np.sin(x) + np.sin(3*x) + np.sin(4*x) + np.sin(15*x) + np.sin(7*x) + .3*np.sin(20*x)

plt.plot(x, y)

#plot labels and style
plt.xlabel('x', fontsize=15) #Label x
plt.ylabel('y', fontsize=15)#Label y

# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.1.2</span>

Consider the formula in the code cell shown below, which shows a function composed of several frequency components. Run the code and view the plot that is generated.

Approximately what are the maximum and minimum values of the total signal amplitude? Enter your answer as a list of numbers `[max,min]` with precision 1e-1.

*Hint: Write some additional code to find the max/min values.*


In [ ]:
#>>>PROBLEM: P3.1.2

import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0,4*np.pi,0.01)   # start,stop,step
y = np.sin(x) + np.sin(4*x) + np.sin(20*x)

plt.plot(x, y)

#plot labels and style
plt.xlabel('x', fontsize=15) #Label x
plt.ylabel('y', fontsize=15)#Label y

# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.1.3</span>

Suppose that the code in Problem 3.1.2 was changed so that the $\sin(4x)$ term became $\sin(3x)$. Would the maximum amplitude be larger or smaller?

*Hint: Try plotting this yourself!*

<a name='section_3_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P3.2 The Discrete Fourier Transform</h2>    

| [Top](#section_3_0) | [Previous Section](#section_3_1) | [Problems](#problems_3_2) | [Next Section](#section_3_3) |


<h3>Overview</h3>

- The decomposition of a signal into a sum of sinusoidal terms can characterized by their frequencies.
- A mathematical method can be used to treat the components of a signal with a given frequency range differently than components at other frequencies. 


<h3>Analysis Using Discrete Fourier Transforms</h3>

We want to figure out how to determine the coefficients $c_{k}$ and $d_{k}$ in the expression of an arbitrary signal in terms of sinusoidal functions (or more exactly, the equivalent coefficients when the $e^{ix}$ notation is used), essentially asking how much a particular frequency contributes to the signal. This can be done by multiplying the signal by a sinusoid of that frequency and then summing that product over all the data in the signal.

Continuous Fourier Transforms can be performed if the "data" is a known continuous function. In that case, the product of the function and the sinusoid would be integrated over all time.

However, measured data always consists of values of some observable at discrete points in time, and therefore the product must be summed in what is called a Discrete Fourier Transform.

The results of this summing over time are called the *Fourier frequency coefficients*:

$$ X[k] = \frac{1}{N}\sum\limits_{n=1}^{N-1} x[n]e^{-i\frac{2\pi k}{N} n} $$ 
<br>

where $x[n]$ are the $N$ discrete points in the data ($x[0]\ldots x[N-1]$) which are multiplied by the sinusoid $e^{-i\frac{2\pi k}{N} n}$ and then summed over all $N$ points to yield the resulting frequency coefficients $X[k]$. Expressed most simply, $X[k]$ is large if sinusoids of frequency $(2\pi k)/N$ contribute strongly to the dataset $x[n]$. Note that while the data $x[n]$ is discrete, the frequency coefficients $X[k]$ are continuous.

<br>

<h4>Note on "Frequency"</h4>

When we use the word "frequency" in this course, we will most often be referring to what is also called the "angular frequency", which is measured in radians per second and denoted using the symbol $\omega$. The more common definition of frequency in terms of cycles per second is usually denoted $f$. The two are related by $f=\omega/(2\pi)$. In the formula shown above, $\omega=(2\pi k)/N$ and $f=k/N$

<h4>Fourier Frequency Coefficients for a Pure Sinusoid</h4>

Suppose that the data can be represented as a single sinusoid with amplitude $1$ and frequency $\omega = (2\pi j)/N$: $x[n]=e^{-i\frac{2\pi j}{N} n}$. 

In this case, not surprisingly, there is only one frequency coefficient:

$$X[k] = \frac{1}{N}\sum\limits_{n=1}^{N-1} (e^{-i\frac{2\pi j}{N} n})(e^{i\frac{2\pi k}{N} n}) = \frac{N}{N}\delta_{jk} = \delta_{jk}$$ 

where the "delta" function $\delta_{jk}$ is $1$ if $j=k$ and zero otherwise.
<br>

<h4>The DC Term</h4>

If the average value of a signal over one period is non-zero, i.e. the data has a constant offset, the signal will have a 0 frequency coefficient $X[0]$. This is also called the "DC" term, by analogy to electrical circuits, where "DC" refers to a constant voltage.

<h3>Synthesis (Inverse Discrete Fourier Transform)</h3>

Given only the Fourier frequency coefficients $X[k]$, one can recompose the signal $x[n]$ using what is called an "Inverse Fourier Transform". As for the transform used to find the frequency coefficients, measured data consisting of the values of an observable at discrete points in time will have an inverse transform consisting of a sum:

$$x[n] = \sum\limits_{k=0}^{N-1}X[k]e^{i\frac{2\pi k}{N} n}$$


In the figures shown below, the data are shown on the left and the frequency coefficients are shown on the right. For simplicity, the data is shown as a continuous line.

<p align="center">
<img alt="Fourer transform" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/box_transform_2.PNG" width="600"/>
</p>


Notice that when we tighten our signal in time (i.e. shorten the duration), the resulting frequency distribution is wider. This is a visual representation of the famous _uncertainty principle_! It arises in quantum mechanics because position and momentum obey the same Fourier relationship that time and frequency do here.

Take a look at the code below which uses numpy's built in `fft` method to take a discrete Fourier transform. Play around with the signal width and qualitatively verify that the uncertainty principle discussed above is valid.

One very important feature of this code is the line:

<pre>
fourierTransform = np.fft.fft(amplitude)/len(amplitude)
</pre>

As the comment indicates, the Fourier transform is divided by `len(amplitude)` in order to normalize it, in other words to ensure that the integral is unity.


In [ ]:
#>>>RUN: P3.2-runcell01

import numpy as np
import matplotlib.pyplot as plt

samplingFrequency   = 100
signal_width = 2

#Plotting amplitude vs. time
x = np.arange(-5, 5, .01)
amplitude = np.piecewise(x, [x < 0-signal_width/2, ((x >= 0-signal_width/2) & (x < 0+signal_width/2)), x >= 0+signal_width/2], [0, 1, 0])
plt.title('Signal across time',fontsize=15)
plt.plot(x, amplitude)
plt.xlabel('Time',fontsize=15)
plt.ylabel('Amplitude',fontsize=15)
# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

#Using np.fft.fft to take the Fourier transform
fourierTransform = np.fft.fft(amplitude)/len(amplitude)    #Divide by length of the amplitude to normalize the transform
fourierTransform = fourierTransform[range(-int(len(amplitude)/2),int(len(amplitude)/2))] #Only look in a select range
   
#parametrize the frequency space
tpCount     = len(amplitude)
values      = np.arange(-int(tpCount/2), int(tpCount/2))
timePeriod  = tpCount/samplingFrequency
frequencies = values/timePeriod

#Plot the frequency space
plt.title('Fourier transform',fontsize=15)
plt.plot(frequencies, abs(fourierTransform))
plt.xlabel('Frequency',fontsize=15)
plt.ylabel('Amplitude',fontsize=15)
# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()


<h3>Spectral Density/Power Spectrums</h3>

The term "spectral density" is used to describe how the content of a signal is distributed over frequency space. There are two complementary versions depending on the nature of the signal.

<h4>Energy Spectral Density (ESD)</h4>

For transient (pulse-like) signals with are localized in time, one can quantify how their total energy is distributed. This is usually represented as simply the square of the amplitude of the Fourier transform as a function of frequency:

$$\overline{S}_{xx}(f) = |{X(f)}|^2$$

<h4>Power Spectral Density (PSD)</h4>

For continuous signals, i.e. ones that extend over all time (or, more realistically over a very long time), the total energy is very large, approaching infinity. In these cases, it is more useful to quantify the spectral density of the power. The power spectral density of a time series is the measure of how the signal's power is distributed across the frequency components that compose that signal.

In order to simplify the math, we define a new quantity called the "windowed signal":

$$x_T(t)=x(t)w_T(t)$$

where the weight $w_T(t)$ is unity within some arbitrary time interval and zero outside that region. The power spectral density is defined as the normalized limit of the energy spectral density for the windowed signal:

$$S_{xx}(f) = \lim_{T \to \infty} \frac{1}{T}|X_T(f)|^2 $$

Alternatively, the PSD can be defined as the Fourier transform of the autocorrelation function $R_{xx}(\tau)$ which can be described informally as how similar the signal is to itself over time. It is defined as:

$$R_{xx}(\tau)=\int_{-\infty}^{\infty} x_T(t-\tau)x_T(t)dt$$

$$S_{xx}(f) = \int_{-\infty}^{\infty} R_{xx}(\tau)e^{-i2\pi f\tau}d\tau$$



<h4>Amplitude Spectral Density (ASD)</h4>

The amplitude spectral density is just the square-root of the PSD.It is useful when the shape of the spectrum is rather constant, since variations in the ASD will then be proportional to variations in the amplitude of the signal itself.


As one example, the plot below shows a typical ASD as measured by the LIGO experiment. Note that the vertical scale is logarithmic in order to show details of the relatively small signals at high frequencies. The frequency axis is also logarithmic in order to expand the structure at low values.

<p align="center">
<img alt="ASD of LIGO data" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/spectrum.png" width="500"/>
</p>

>source: Project 1

As you can see, the overall spectrum is dominated by a steep rise at low frequencies and then there are narrow spectral peaks throughout. Almost all of the structure that you see in this plot, both the overall shape and the peaks, arises from artifacts in the instrument itself (also called background sources) such as seismic noise and "violin modes" from the suspension fibers of the LIGO mirrors. 

As a side note, the peaks are often called "lines", a reference to viewing light passed through a diffraction grating, where regions of high intensity would create distinctive lines when the spectrum was projected onto a screen. 

<a name='problems_3_2'></a>     

| [Top](#section_3_0) | [Restart Section](#section_3_2) | [Next Section](#section_3_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.2.1</span>

Consider the `amplitude` function shown in the code below. it consists of a simple sum (or "mixing") of three different sinusoidal functions. Using `np.fft.fft`, translate this amplitude distribution from its time domain to its frequency domain. Make sure to normalize the Fourier transform (as done in the previouis code segment in this section).

What is the height of the tallest peak in the frequency space? Enter your answer as a number with precision 1e-1.

In [ ]:
#>>>PROBLEM: P3.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

samplingFrequency   = 100
samplingInterval = 1 / samplingFrequency

beginTime = 0
endTime = 10

time = np.arange(beginTime, endTime, samplingInterval);

amplitude1 = np.sin(2*np.pi*23*time)
amplitude2 = 3*np.sin(2*np.pi*17*time)
amplitude3 = -1.5*np.sin(2*np.pi*13*time)

amplitude = amplitude1 + amplitude2 + amplitude3
plt.title('Sine wave with multiple frequencies',fontsize=15)
plt.plot(time, amplitude)
plt.xlabel('Time',fontsize=15)
plt.ylabel('Amplitude',fontsize=15)
# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

##############################################
############## INSERT CODE HERE ##############
##############################################


##############################################
##############################################

plt.title('Fourier transform depicting the frequency components',fontsize=15)
plt.grid(color='grey', linestyle='-', linewidth=1)
plt.plot(frequencies, abs(fourierTransform))
plt.xlabel('Frequency',fontsize=15)
plt.ylabel('Amplitude',fontsize=15)
# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.2.2</span>

Compute the ESD and approximate the PSD and ASD of the signal consisting of the sum of three sinusoids as shown in the previous problem. What are the values of the ESD, PSD, and ASD at the frequency $f_0=17$ Hz?

Use the following definitions:

<pre>
ESD = np.abs(fourierTransform)**2
PSD = ESD/endTime
ASD = np.sqrt(PSD)
</pre>

Enter your answer as a list of numbers `[ESD(f0), PSD(f0), ASD(f0)]` with precision 1e-2.

<a name='section_3_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P3.3 Spectrogram and Q-Transform</h2>   

| [Top](#section_3_0) | [Previous Section](#section_3_2) | [Problems](#problems_3_3) |


<h3>Overview</h3>

- To better understand a system, we can bridge the gap between the frequency and time domains with another transform.
- The Short-Time Fourier Transform takes the Fourier transform within shorter time segments. 
- The resulting visual representation is called a "spectrogram" which can be thought of as a series of Fourier transforms stacked on their side. 

The figure below shows an example of such a spectrogram. You can clearly see that the dominant frequencies (the bright lines) move around as a function of time. 

<p align="center">
<img alt="spectrogram example 1" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/motif_sgram.png" width="500"/>
</p>

The Q-transform breaks these intervals up with logarithmic spacing, which is particularly useful for audio signals (and other signals, as we will see). Below we see an example of the difference between a spectrogram and a Q-transform.


<p align="center">
<img alt="spectrogram example 2" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/stft_vs_q_gautham.png" width="600"/>
</p>

>source: https://ccrma.stanford.edu/~gautham/Site/Multipitch.html <br>
>attribution: Stanford

<h3>Whitening data</h3>

To better visualize deviations from the noise, it is useful to employ a technique called "whitening". This procedure takes the data and attempts to make the power spectral density flat (i.e. normalize the power at all frequencies) so that excess power at any frequency is more obvious. Persistent signals like noise will have their power spread across the entire time window, while any localized signals will have all of their power in one region of time and will show up as clear spikes.

Whitening can be achieved (roughly speaking) by applying the inverse frequency response of the raw signal. 

Example: Transmitted power in one of the interferometer arms with two large glitches with a frequency around 5-50Hz at times of 11.5 and 17 seconds.

<p align="center">
<img alt="LIGO data whitened" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/whiteningexample_gwpy.png" width="800"/>
</p>

>source: Project 1

<h3>Filtering</h3>

Filtering refers to the attenuation of the components of a signal in a particular range of frequencies. This has the complementary effect of amplifying the components at other frequencies once the filtered spectrum is renormalized. While filtering can, in principle, be done in the time domain, is best performed within the frequency domain. 

The effect of a particular filter is usually represented as a set of scale factors in the frequency domain. Also called the frequency response of the filtering system, the frequency spectrum is multiplied by these factors to produce the modified output spectrum.
    
The figure below shows examples of such scale factors as a function of frequency. The names that are applied to these various examples describe the effect of suppressing certain frequencies (the areas where the scale factors go to zero). Because the output is what matters, the names of such filters typically describe the frequencies that "pass" through the filter, i.e. those that are not suppressed. 

    
<p align="center">
<img alt="bandpass examples" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/filters.png" width="800"/>
</p>

<h3>Example with Sound File</h3>

We are going to work through an example of filtering using an audio file. First, we will import the relevant libraries and then download a free file containing the sound to be filtered.

To read more about soundfile and sounddevice, see here:

- https://pysoundfile.readthedocs.io/en/latest/
- https://python-sounddevice.readthedocs.io/en/0.4.5/


**Note: If you are running this Jupyter notebook locally, sound files may not play in all browsers. You may encounter issues in Safari, for instance.**

See the source and attribution information for the audio file below:


>data: data/P03/track.flac <br>
>source: https://freesound.org/s/647536/ <br>
>attribution: Bertsz <br>
>license type: This work is licensed under the Creative Commons 0 License.

In [ ]:
#>>>RUN: P3.3-runcell01

import time
import soundfile as sf
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Audio, display

#first we load a sound from this link here https://freesound.org/s/647536/

data, samplerate = sf.read('data/P03/track.flac')

<h4>Amplitude and Frequency Manipulation</h4>

Working through the code cells below, we will do the following:

- Play the original file for 2 seconds
- Play a note at 440 Hz
- Fourier transform the 440 Hz note and plot the frequency spectrum
- Decrease the amplitude of the 440 Hz note in Fourier space, convert back to the time domain, and play the result
- Shift the frequency of the 440 Hz note in Fourier space (converting it to 880 Hz), convert back to the time domain, and play

The `play()` function used in these code cells will create a small pop-up window. Click the arrow to play the sound. After the sound output finishes, just click the arrow again to replay it. If you want, you can actually run two or more of these sounds at the same time. To avoid an excessively loud sound, you may want to turn down your audio output the first time you play one of these results. 

In [ ]:
#>>>RUN: P3.3-runcell02

#play the original file
#the original file is 60s

def play(iArray,iFS):
    sf.write('data/P03/tmp.flac', iArray, iFS)
    display(Audio('data/P03/tmp.flac',autoplay=False))


play(data,samplerate)

In [ ]:
#>>>RUN: P3.3-runcell03

#play the original file for 2 seconds
#the original file is 60s

sec=2
min=0
max=int((sec/60.)*len(data))
play(data[min:max],samplerate)

In [ ]:
#>>>RUN: P3.3-runcell04

#play a note at 440 Hz
sec=2
timeseq = np.arange(0,sec,1./samplerate);

#let's do an A at 440Hz
a440 = np.sin(2*np.pi*440*timeseq)
play(a440,samplerate)

In [ ]:
#>>>RUN: P3.3-runcell05

#Fourier transform the 440 Hz note and plot the frequency spectrum
fta440 = np.fft.fft(a440)

values      = np.arange(int(len(a440)))
timePeriod  = len(a440)/samplerate
frequencies = values/timePeriod
plt.plot(frequencies[0:2000],np.abs(fta440[0:2000]))
plt.show()

In [ ]:
#>>>RUN: P3.3-runcell06

#decrease the amplitude of the 440 Hz note in Fourier space,
#convert back to the time domain and play
ifta440 = np.fft.ifft(fta440*0.1)

play(ifta440.real,samplerate)

In [ ]:
#>>>RUN: P3.3-runcell07

#shift the frequency of the 440 Hz note in Fourier space (converting to 880 Hz),
#convert back to the time domain and play
fta880  = fta440
fta880[880*2] = fta440[440*2]
fta880[440*2] = 0
ifta880 = np.fft.ifft(fta880)

play(ifta880.real,samplerate)

<h4>Bass Boosting</h4>

Now we will manipulate the data in Fourier space to increase the amplitude of the lower frequencies. We first plot the result of this manipulation in the frequency domain.

Note that in the following plots, the before signal is plotted as light blue, the after signal is plotted as light peach, and anyplace where the two overlap is a mixture of both colors.

In [ ]:
#>>>RUN: P3.3-runcell08

length_sec=len(data)/samplerate
fc = 400 #cutoff frequency in Hz; boost all frequency content below fc
kc = int(fc*length_sec)

X = np.fft.fft(data[:,0])
Y = X.copy() #copy values from X
#boost the bass
Y[:kc]    = [i*10 for i in X[:kc]]

def plotFFT(X,Y,data,samplerate):
    values      = np.arange(int(len(data[:,0])))
    timePeriod  = len(data[:,0])/samplerate
    frequencies = values/timePeriod
    plt.plot(frequencies[0:int(length_sec*2000)],np.abs(X[0:int(length_sec*2000)]),alpha=0.5,label='before')
    plt.plot(frequencies[0:int(length_sec*2000)],np.abs(Y[0:int(length_sec*2000)]),alpha=0.5,label='after')
    plt.ylabel('amp')
    plt.xlabel('freq')
    plt.legend() 
    plt.yscale('log')
    plt.show()

plotFFT(X,Y,data,samplerate)

Now we take the inverse Fourier transform and play the audio to compare with the original.

In [ ]:
#>>>RUN: P3.3-runcell09

## Inverse Fourier transform and recombine into a .wav file 
y = np.fft.ifft(Y)

#start the file at 30 sec
def replay(y,data,samplerate=samplerate,playtime=10):
    print("play 1st: altered waveform")
    play(y[30*samplerate:(30+playtime)*samplerate].real,samplerate)
    
    print("play 2nd: original")
    #and for comparison
    play(data[30*samplerate:(30+playtime)*samplerate].real,samplerate)


replay(y,data)

<h4>Bass Isolation (Low Pass)</h4>

Finally, we use a low pass filter to truncate the data in the frequency domain. Again, the plot below shows the waveform in Fourier space, and then the time domain data are played.

In [ ]:
#>>>RUN: P3.3-runcell10

fc = 1000 #cutoff frequency in Hz; boost all frequency content below fc
kc = int(fc*length_sec)

X = np.fft.fft(data[:,0])
Y = X.copy() #copy values from X
Y[:kc]    = [i*10 for i in X[:kc]]
Y[kc:]    = [i*0 for i in X[kc:]]
y = np.fft.ifft(Y)

plotFFT(X,Y,data,samplerate)
replay(y,data,samplerate)

<h4>Signal Detection and Matched Filtering</h4>

- If we have an unknown, noisy signal we can try to detect the presence of a known signal with matched filtering. 
- If we *know* or *guess* the signal we're looking for (called the *template*), we can use it as a filter for combing the data for the presence of that template.
- Matched filters work by maximizing the signal to noise ratio (SNR) when the matched filter detects the presence of the template signal in a noisy signal. 
- We will talk more about convolutions next week but we can informally think of a matched filter as:
- "Drag" or sweep your template across the signal and calculate some statistic.
- The optimal statistic suggests the presence of a signal. 

<p align="center">
<img alt="whitened data around event" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/template.png" width="700"/>
</p>
<p align="center">
<img alt="matched filtering" src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/SNR.png" width="700"/>
</p>

There are many features in our example data that we might want to highlight. Lets take a look at a specific frequency range and see if we can find some features. Let's plot between 500 and 600 Hz. 

In [ ]:
#>>>RUN: P3.3-runcell11

dft_cof = np.fft.fft(data[:,0])
values      = np.arange(int(len(data[:,0])))
timePeriod  = len(data[:,0])/samplerate
frequencies = values/timePeriod
minHz=500
maxHz=600
plt.plot(frequencies[int(length_sec*minHz):int(length_sec*maxHz)],np.abs(X[int(length_sec*minHz):int(length_sec*maxHz)]),alpha=0.5)
plt.ylabel('amp')
plt.xlabel('freq')
plt.show()


Hmm, I wonder if we can isolate the peak region and hear something. Let's do +/- 30 Hz around. Do you hear something? What about +/- 130 Hz around? 

In [ ]:
#>>>RUN: P3.3-runcell12

#now let's isolate a frequency
minHz=0
maxHz=int(len(dft_cof)/int(length_sec))
isofreq=550
freqrange=30 #130
dft_cof = np.fft.fft(data[:,0])
norm=np.sum(dft_cof)
for tmpfreq in range(minHz*int(length_sec),maxHz*int(length_sec)):
    if (float(tmpfreq) < (isofreq-freqrange)*length_sec) or (float(tmpfreq) > (isofreq+freqrange)*length_sec):
        dft_cof[tmpfreq] *= 0
        
scale=np.sum(dft_cof)/norm #CHANGE THE SCALE TO INCREASE THE AMPLITUDE IF NEEDED
dft_cof*=scale
plotFFT(X,dft_cof,data,samplerate)
isofreqdata = np.fft.ifft(dft_cof)
x           = np.fft.ifft(X)

replay(isofreqdata,x)

<a name='problems_3_3'></a>   

| [Top](#section_3_0) | [Restart Section](#section_3_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.3.1</span>


Now, we are going to use the same sound file and modify the song so that the amplitude across each range is exactly the same. This a variation of what we call whitening. 

The idea will be that we go into frequency space and the compute the amplitude $A=\sqrt{\overline{S}_{xx}(f)} = \sqrt{|{X(f)}|^2}$. We can then take our sample in frequency space and divide by the amplitude over all of frequency space. We call this `dft_cof_adjusted` in the code below. Finally, we can take the inverse Fourier transform and listen to the result.


Complete the code below to calculate `dft_cof_adjusted`, and achieve the objectives outlined above. What does the processed file sound like?  (Note will will have to rescale the normalized sound by a constant so you can hear it).

Choose from the options below:

- The processed file sounds much clearer, as all of the noise has been removed.
- There is a lot of high freqeuncy noise now because we increased the high frequency amplitudes and lowered the low frequency.
- There is a lot of low frequeuncy noise now because we increased the low freqency amplitudes and increased the high frequency.


In [ ]:
#>>>PROBLEM: P3.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

length_sec=len(data)/samplerate
dft_cof = np.fft.fft(data[:,0])

dft_cof_adjusted=#Insert code here 

#Not you may need to scale amplitde with line below
#dft_cof_adjusted*=500

plotFFT(X,dft_cof_adjusted,data,samplerate)
isofreqdata = np.fft.ifft(dft_cof_adjusted)
X           = np.fft.fft(data[:,0])
x           = np.fft.ifft(X)
replay(isofreqdata,x,playtime=16)


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.3.2</span>

Now inject sine waves raning from  440 Hz to 880 Hz with a spacing of 50Hz into the data, by adding these to the signal. Use the amplitdue of 0.01 as specified in the 440 Hz example below. Our objective is to compare how well we can hear the injected signal in the data before and after whitening. Does whitening enable us to hear the injected signal more clearly?

Whiten the dataset with the injected signal using the the amplitude $A$ of the non-injected, raw data. Call this whitened data `dft_cof_inj_n`.

After whitening the data, run the function `replay_inj` and listen to the sample before and after whitening. Do you hear the added sine waves in either sound sample? What about if you increase the amplitude to 0.1? 

Select the option below that best describes the effect of whitening:

- Whitening makes the signal clear over the background noise.
- Whitening smooths both the original data and signal so that the two are indistinguishable.
- Whitening has no affect on the injected signal.


In [ ]:
#>>>PROBLEM: P3.3.2

length_sec=len(data)/samplerate
raw_data      = data[:,0].copy()
data_injected = data[:,0].copy()
timeseq = np.arange(0,length_sec,1./samplerate);

amp=0.01 #TRY CHANGING THIS VALUE TO 0.1

#example injection for 440
a440 = (np.sin(2*np.pi*440*timeseq+np.pi))*amp
data_injected+=a440

#YOUR CODE HERE
#WRITE A LOOP TO ADD THE SIGNALS IN THE FREQUENCY RANGE range(440,880,50)

dft_cof     = np.fft.fft(raw_data)
dft_cof_inj = np.fft.fft(data_injected)
#dft_cof_inj_n = #YOUR CODE HERE

#Note you will have to scale up the volume again
#dft_cof_inj_n*=500

#Inverse fourier transform
isofreqdata = np.fft.ifft(dft_cof_inj_n)

#Now Plot it and Play it 
plotFFT(dft_cof_inj,dft_cof_inj_n,data,samplerate)

#DEFINING A FUNCTION TO COMAPRE DATA WITH INJECTED SIGNAL
def replay_inj(y,data,samplerate=samplerate,playtime=10):
    print("play 1st: data with injected signal")
    play(data[30*samplerate:(30+playtime)*samplerate].real,samplerate)
    
    print("play 2nd: whitened data with injected signal")
    #and for comparison
    play(y[30*samplerate:(30+playtime)*samplerate].real,samplerate)
    
replay_inj(isofreqdata,data_injected,playtime=16)
